# Día 4: Desplegando el contrato

## Objetivo
Aprender a desplegar un Smart Contract en la blockchain local usando Python y Web3.py.

## 1. Preparación para el despliegue

Primero, necesitamos cargar el bytecode y el ABI del contrato que compilamos en el día anterior. Si no tienes estos archivos, ejecuta primero el notebook del Día 3.

In [ ]:
# Importamos las bibliotecas necesarias
from web3 import Web3
import json
from solcx import compile_standard, install_solc

# Instalamos el compilador si es necesario
install_solc('0.8.0')

# Leemos y compilamos el contrato
with open('SimpleStorage.sol', 'r') as file:
    simple_storage_file = file.read()

compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SimpleStorage.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
            }
        },
    },
    solc_version="0.8.0",
)

# Extraemos el bytecode y el ABI
bytecode = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["evm"]["bytecode"]["object"]
abi = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["abi"]

print("Contrato compilado y listo para desplegar.")

## 2. Conexión a la blockchain

Nos conectamos a nuestra blockchain local (Ganache):

In [ ]:
# Conectamos con Ganache
web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))

# Verificamos la conexión
if web3.is_connected():
    print("Conectado a Ganache correctamente.")
    print(f"Bloque actual: {web3.eth.block_number}")
else:
    print("Error: No se pudo conectar a Ganache. Asegúrate de que esté en ejecución.")

## 3. Creación del objeto de contrato

Creamos un objeto de contrato usando el bytecode y el ABI:

In [ ]:
# Creamos el objeto del contrato
SimpleStorage = web3.eth.contract(abi=abi, bytecode=bytecode)
print("Objeto de contrato creado.")

## 4. Despliegue del contrato

El despliegue de un contrato se realiza en dos pasos:

1. Construir la transacción de despliegue
2. Enviar la transacción y esperar la confirmación

In [ ]:
# Obtenemos la cuenta que usaremos para desplegar el contrato
cuenta = web3.eth.accounts[0]
print(f"Usando la cuenta: {cuenta}")

# Obtenemos el nonce (número de transacciones enviadas por la cuenta)
nonce = web3.eth.get_transaction_count(cuenta)

# 1. Construimos la transacción de despliegue
tx = SimpleStorage.constructor().build_transaction(
    {
        "from": cuenta,
        "nonce": nonce,
        "gas": 2000000,  # Límite de gas
        "gasPrice": web3.to_wei("50", "gwei")  # Precio del gas
    }
)

# 2. Firmamos la transacción (en Ganache no es necesario, pero lo incluimos para completitud)
# En una red real, necesitaríamos la clave privada: tx_signed = web3.eth.account.sign_transaction(tx, private_key)

# 3. Enviamos la transacción
tx_hash = web3.eth.send_transaction(tx)
print(f"Transacción enviada: {tx_hash.hex()}")

# 4. Esperamos la confirmación
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Contrato desplegado en la dirección: {tx_receipt.contractAddress}")

## 5. Guardando la dirección del contrato

Es importante guardar la dirección del contrato para poder interactuar con él más tarde:

In [ ]:
# Guardamos la dirección del contrato en un archivo
contract_address = tx_receipt.contractAddress
with open('contract_address.txt', 'w') as file:
    file.write(contract_address)

print(f"Dirección del contrato guardada en 'contract_address.txt': {contract_address}")

## 6. Entendiendo los componentes clave

### ABI (Application Binary Interface)
- Es la interfaz que define cómo interactuar con el contrato
- Describe las funciones, eventos y sus parámetros
- Es necesario para llamar a las funciones del contrato

### Bytecode
- Es el código ejecutable que se despliega en la blockchain
- Resultado de compilar el código Solidity
- Se ejecuta en la Máquina Virtual de Ethereum (EVM)

### Dirección del contrato
- Identifica de manera única al contrato en la blockchain
- Se genera durante el despliegue
- Es necesaria para interactuar con el contrato desplegado

## 7. Verificando el despliegue

Podemos verificar que nuestro contrato se ha desplegado correctamente creando una instancia del contrato con la dirección:

In [ ]:
# Creamos una instancia del contrato desplegado
simple_storage = web3.eth.contract(address=contract_address, abi=abi)

# Verificamos que podemos llamar a la función get()
valor_almacenado = simple_storage.functions.get().call()
print(f"Valor almacenado actualmente: {valor_almacenado}")

print("¡El contrato se ha desplegado correctamente y está funcionando!")

## Lo aprendido hoy

- Preparamos un contrato para su despliegue en la blockchain
- Desplegamos el contrato usando Web3.py
- Entendimos los componentes clave: ABI, bytecode y dirección del contrato
- Verificamos que el contrato se desplegó correctamente

En el próximo día, aprenderemos a interactuar con el contrato desplegado para leer y escribir datos.